# Multi-Layer Neural Network

  We're implementing a multi layer neural network for animal classification. Our network will consist of one input layer, n hidden layers and one output layer.

In [122]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
np.random.seed(35819)

In [124]:
def generate_input_layer(image_path: str, input_layer_size: int=2500):
    dim = int(np.sqrt(input_layer_size))
    image = Image.open(image_path).resize((dim, dim))
    image = ImageOps.grayscale(image)
    imdata = np.asarray(image)
    imdata -= int(np.mean(imdata))
    imdata = (imdata - np.min(imdata)) / (np.max(imdata) - np.min(imdata)) if (np.max(imdata) - np.min(imdata)) != 0 else imdata-imdata # squishing integer vals between 0-255 into 0-1
    return imdata.flatten()

 Our true label should be in the format of one hot encoding in order to compare to the predicted value easily therefore we're converting them.

In [125]:
classes = ["dog", "horse", "elephant", "butterfly", "chicken", "cat", "cow", "sheep", "spider", "squirrel"]
def one_hot(label):
    onehot = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(10):
        if label == classes[i]: onehot[i] = 1
    return onehot

 We're reading the images and creating the train and test sets.

In [126]:
images = []
labels = []
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "ragno": "spider", "scoiattolo": "squirrel", "dog": "cane", "horse": "cavallo", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo", "sheep": "pecora"}
for filename in os.listdir("archive/raw-img/"):
    if filename == ".DS_Store": continue
    filepath = "archive/raw-img/"+filename+"/"
    for image in os.listdir(filepath):
        inp = generate_input_layer(filepath+image)
        images.append(inp)
        labels.append(one_hot(translate[filename]))

In [127]:
images = np.array(images)
labels = np.array(labels)
images = np.concatenate((images, labels), axis=1)
np.random.shuffle(images)
trainingSet, testSet = train_test_split(images, test_size=0.2, random_state=42)

We're initializing the weights randomly. 

In [128]:
def initialize_weights(inputLayerSize, hiddenLayerNo, hiddenLayerSize, outputLayerSize):
    layer = list()
    for i in range(hiddenLayerNo+1):
        if hiddenLayerNo == 0: 
            weight = np.random.randn(inputLayerSize, outputLayerSize) * np.sqrt(2.0 / (inputLayerSize + outputLayerSize))
        elif i == 0: 
            weight = np.random.randn(inputLayerSize, hiddenLayerSize) * np.sqrt(2.0 / (inputLayerSize + hiddenLayerSize))
        elif i == hiddenLayerNo: 
            weight = np.random.randn(hiddenLayerSize, outputLayerSize)  * np.sqrt(2.0 / (hiddenLayerSize + outputLayerSize))
        else: 
            weight = np.random.randn(hiddenLayerSize, hiddenLayerSize)  * np.sqrt(2.0 / (hiddenLayerSize + hiddenLayerSize))
        layer.append({"W": weight})
    return layer

Here our activations functions to help the network learn complex patterns in the data, softmax function that used for the output layer of neural network models that predict a multinomial probability distribution, loss function to calculate the error and their derivatives.

In [129]:
def sigmoid(x):
	return 1.0 / (1.0 + np.exp(-x))

def sigmoid_deriv(S, grad): #S is output of sigmoid function
    return S * (1 - S) * grad

def reLU(x):
    return np.maximum(0, x)

def reLU_deriv(S, grad): #S is output of reLU function
    return (S > 0) * 1 * grad

def tanH(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def tanH_deriv(S, grad): #S is output of tanH function
    return (1 - np.square(S)) * grad
        
def softmax(vector):
    e = np.exp(vector)
    return e / e.sum(axis=1, keepdims=True)

def softmax_deriv(SM, grad): #SM is output of softmax function
    return (np.sum(grad * -SM, axis=1, keepdims=True)+grad) * SM

def linear_derivX(grad, index, layer): #derivative of loss w.r.t X calculated for the chain rule
    dX = np.dot(grad, np.transpose(layer[index]["W"]))
    return dX

def linear_deriv(grad, index, layer):
    layer[index]["dW"] = np.dot(np.transpose(layer[index-1]["X"]), grad)
    layer[index]["db"] = grad.sum(axis=0)
    return layer

def get_loss(batch, layer):
    return np.sum(np.dot(-np.log(layer[-1]["SM"]), np.transpose(batch[:,2500:])))

def nll_deriv(labels, predictions):
    return -labels/predictions

def bias(size, layerNo, layer):
    layer[layerNo]["b"] = np.full((size,), 0)
    return layer[layerNo]["b"]

def set_biases(layer, hiddenLayerSize):  #initializing bias arrays with 0s for each layer
    for l in range(len(layer)):
        if l == len(layer)-1:
            bias(10, l, layer)
        else:
            bias(hiddenLayerSize, l, layer)
    return layer

 Here we calculate the X values for each layer. Then pass them to activation functions and finally in the output layer, to softmax function.

In [130]:
def get_activations(batch, sig, relu, layer):
    for i in range(1, len(layer)):
        if i == 1:
            X = np.add(np.dot(batch[:,:2500], layer[i]["W"]), layer[i]["b"]) #Oi=WijXj+bi
        elif i == len(layer)-1:
            X = np.add(np.dot(layer[i-1]["S"], layer[i]["W"],), layer[i]["b"])
        else:
            X = np.add(np.dot(layer[i-1]["S"], layer[i]["W"]), layer[i]["b"]) 
        layer[i]["X"] = X
        if sig: 
            layer[i]["S"] = sigmoid(X)
        elif relu:
            layer[i]["S"] = reLU(X)
        else:
            layer[i]["S"] = tanH(X)
    layer[i]["SM"] = softmax(layer[i]["S"])
    return layer

Here the derivative of loss with respect to the weights is calculated by using the chain rule. The chain rule enables the network to identify how much each weight contributes to the error and how much each weight needs to be adjusted. Also derivative of loss w.r.t biases is calculated to update them.

In [131]:
def gradient(hiddenLayerCount, batch, sigmoid, relu, layer):
    func = sigmoid_deriv if sigmoid else reLU_deriv
    func = reLU_deriv if relu else tanH_deriv
    grad = nll_deriv(batch[:,2500:], layer[-1]["SM"])
    grad = softmax_deriv(layer[-1]["SM"], grad)
    for i in range(hiddenLayerCount+1):
        grad = func(layer[hiddenLayerCount-i+1]["S"], grad)
        layer = linear_deriv(grad, hiddenLayerCount-i+1, layer)
        grad = linear_derivX(grad, hiddenLayerCount-i+1, layer)
    return layer

We multiply the derivatives of the loss w.r.t weight and biases by learning rate and the weights and biases are updated accordingly.

In [132]:
def update(hiddenLayerCount, learningRate, layer):
    for i in range(-1, hiddenLayerCount):
        layer[hiddenLayerCount-i]["W"] =  np.subtract(layer[hiddenLayerCount-i]["W"], learningRate * layer[hiddenLayerCount-i]["dW"])
        layer[hiddenLayerCount-i]["b"] = np.subtract(layer[hiddenLayerCount-i]["b"], learningRate * layer[hiddenLayerCount-i]["db"])
    return layer

Here we multiplied the learning rate by some decay rate for it to converge better.

In [133]:
def train(batchSize, epochNo, hiddenLayerCount, learningRate, inputLayerSize, sig, relu):
    hiddenLayerSize = int(2*(inputLayerSize+10)/3)
    layer = [{}]+initialize_weights(inputLayerSize, hiddenLayerCount, hiddenLayerSize, 10)
    imageBatches = np.split(trainingSet, np.arange(batchSize, len(trainingSet), batchSize))
    imageBatches = np.delete(imageBatches, len(imageBatches)-1, 0)
    set_biases(layer, hiddenLayerSize)
    initial = learningRate
    for batch in imageBatches:
        learningRate = initial
        layer = get_activations(batch, sig, relu, layer)
        layer[0]["X"] = batch[:,:inputLayerSize]
        for epoch in range(epochNo):
            learningRate =(1/(1+10*epoch))*learningRate
            gradient(hiddenLayerCount, batch, sig, relu, layer)
            layer = update(hiddenLayerCount, learningRate, layer)
    return layer

Here we are converting the output into one hot encoding by taking the maximum value in the predictions and classify the test as of that class.

In [134]:
def to_onehot(output):
    ind = np.argmax(output, axis=1)
    onehot = np.zeros((len(ind), 10))
    for i in range(len(ind)):
        onehot[i,ind[i]] = 1
    return onehot

In [135]:
def test(batchSize, inputLayerSize, sig, relu, epochNo, hiddenLayerCount, learningRate):
    layer = train(batchSize, epochNo, hiddenLayerCount, learningRate, 2500, sig, relu)
    testBatches = np.split(trainingSet, np.arange(batchSize, len(trainingSet), batchSize))
    testBatches = np.delete(testBatches, len(testBatches)-1, 0)
    totalAccuracy = 0
    for testBatch in testBatches:
        layer = get_activations(testBatch, sig, relu, layer)
        predictions = to_onehot(layer[-1]["SM"])
        accuracy = predictions * testBatch[:, inputLayerSize:]
        accuracy = np.sum(accuracy)/accuracy.shape[0]*100
        totalAccuracy += accuracy
    return totalAccuracy/len(testBatches)

In [136]:
from tabulate import tabulate
learningRates = [0.005, 0.01, 0.02]
batchSizes = [16, 64, 128]
hiddenUnitNos = [0, 1, 2]

 Even though we used a decay parameter, the values given were too large and they didn't converge. Therefore all of them has the same accuracy and don't give us much information. For the next trainings we used a smaller learning rate which is 0.0001 and got better results.

In [154]:
learningRateAccuracies = [["Learning Rate", "Accuracy"], [learningRates[0], test(128, 2500, 0, 0, 10, 2, learningRates[0])], [learningRates[1], test(128, 2500, 0, 0, 10, 2, learningRates[1])],[learningRates[2], test(128, 2500, 0, 0, 10, 2, learningRates[2])]]
print(tabulate(learningRateAccuracies, headers='firstrow', tablefmt='fancy_grid'))

/var/folders/d6/p_p19vt54p1gb7kxxflt5tk00000gn/T/ipykernel_26477/3370038240.py:14: RuntimeWarning: overflow encountered in exp
  return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))
/var/folders/d6/p_p19vt54p1gb7kxxflt5tk00000gn/T/ipykernel_26477/3370038240.py:14: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))


╒═════════════════╤════════════╕
│   Learning Rate │   Accuracy │
╞═════════════════╪════════════╡
│           0.005 │    18.7069 │
├─────────────────┼────────────┤
│           0.01  │    18.7069 │
├─────────────────┼────────────┤
│           0.02  │    18.7069 │
╘═════════════════╧════════════╛


 As batch size increases, the number of iterations decrease which means error will be minimized less times. Therefore selecting a smaller batch size gives us better accuracy.

In [148]:
batchSizeAccuracies = [["Batch Size", "Accuracy"], [batchSizes[0], test(batchSizes[0], 2500, 0, 1, 10, 2, 0.0001)], [batchSizes[1], test(batchSizes[1], 2500, 0, 1, 10, 2, 0.0001)],[batchSizes[2], test(batchSizes[2], 2500, 0, 1, 10, 2, 0.0001)]]
print(tabulate(batchSizeAccuracies, headers='firstrow', tablefmt='fancy_grid'))

╒══════════════╤════════════╕
│   Batch Size │   Accuracy │
╞══════════════╪════════════╡
│           16 │    26.5721 │
├──────────────┼────────────┤
│           64 │    26.5243 │
├──────────────┼────────────┤
│          128 │    26.4571 │
╘══════════════╧════════════╛


 If the data is linearly separable we don't need any hidden layers at all. Other than that, one hidden layer is sufficient for the large majority of problems. Problems that require two hidden layers are rarely encountered. However, neural networks with two hidden layers can represent functions with any kind of shape. In our case, 2 layers worked better.

In [149]:
hiddenUnitAccuracies = [["Hidden Unit No", "Accuracy"], [hiddenUnitNos[0], test(128, 2500, 0, 1, 10, hiddenUnitNos[0], 0.0001)], [hiddenUnitNos[1], test(128, 2500, 0, 1, 10, hiddenUnitNos[1], 0.0001)],[hiddenUnitNos[2], test(128, 2500, 0, 1, 10, hiddenUnitNos[2], 0.0001)]]
print(tabulate(hiddenUnitAccuracies, headers='firstrow', tablefmt='fancy_grid'))

╒══════════════════╤════════════╕
│   Hidden Unit No │   Accuracy │
╞══════════════════╪════════════╡
│                0 │    21.8558 │
├──────────────────┼────────────┤
│                1 │    21.3765 │
├──────────────────┼────────────┤
│                2 │    26.5194 │
╘══════════════════╧════════════╛


 In some cases, the gradient will be vanishingly small, effectively preventing the weight from changing its value. For sigmoid and tanH functions, this vanishing gradient problem makes the learning process very slow and make them converge to their optimum. However for reLU function, maximum threshold is infinity and there is no issue of vanishing gradient problem, it shows better convergence therefore the accuracy is maximum.

In [150]:
ActivationFunctionAccuracies = [["Activation Function", "Accuracy"], ["Sigmoid", test(128, 2500, 1, 0, 10, 2, 0.0001)], ["ReLU", test(128, 2500, 0, 1, 10, 2, 0.0001)],["tanH", test(128, 2500, 0, 0, 10, 2, 0.0001)]]
print(tabulate(ActivationFunctionAccuracies, headers='firstrow', tablefmt='fancy_grid'))

╒═══════════════════════╤════════════╕
│ Activation Function   │   Accuracy │
╞═══════════════════════╪════════════╡
│ Sigmoid               │    19.9195 │
├───────────────────────┼────────────┤
│ ReLU                  │    25.2109 │
├───────────────────────┼────────────┤
│ tanH                  │    18.2947 │
╘═══════════════════════╧════════════╛
